# FastHTML By Example

> An alternative introduction



There are lots of non-FastHTML-specific tricks and patterns involved in building web apps. The goal of this tutorial is to give an alternate introduction to FastHTML, building out example applications to show common patterns and illustrate some of the ways you can build on top of the FastHTML foundations to create your own custom web apps. A secondary goal is to have this be a useful document to add to the context of an LLM to turn it into a useful FastHTML assistant - in fact, in some of the examples we'll see this kind of assistant in action, thanks to [this custom GPT](https://chatgpt.com/g/g-xPqF9SZjM-fasthtml-helper).

Let's get started.

## FastHTML Basics

FastHTML is *just python*. You can install it with `pip install python-fasthtml`, and extensions/components built for it can likewise be distriuted via pypi or as simple python files.

The core usage of FastHTML is to define routes, and then to define what to do at each route. This is similar to the [FastAPI](https://fastapi.tiangolo.com/) web framework (in fact we implemented much of the fuctionality to match the FastAPI usage examples) but where FastAPI focuses on returning JSON data to build APIs, FastHTML focuses on returning HTML data.

Here's a simple FastHTML app that returns a "Hello, World" message:

In [ ]:
from fasthtml import FastHTML

app = FastHTML()

@app.get("/")
def home():
    return "<h1>Hello, World</h1>"

To run this app, place it in a file, say `app.py`, and then run it with `uvicorn app:app --reload`. You'll see a message like this:
```
INFO:     Will watch for changes in these directories: ['/home/jonathan/fasthtml-example']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [871942] using WatchFiles
INFO:     Started server process [871945]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

If you navigate to http://127.0.0.1:8000 in a browser, you'll see your "Hello, World". If you edit the `app.py` file and save it, the server will reload and you'll see the updated message when you refresh the page in your browser.

## Constructing HTML

Notice we wrote some HTML in the previous example. We don't want to do that! Some web frameworks require that you learn HTML, CSS, Javascript AND some templating language AND python. We want to do as much as possible with just one language. Fortunately, [fastcore.xml](https://fastcore.fast.ai/xml.html) has all we need for constructing HTML from python, and FastHTML includes all the tags you need to get started. For example:

In [ ]:
from fasthtml.common import *
page = Html(
    Head(Title('Some page')),
    Body(Div('Some text, ', A('A link', href='https://example.com'), Img(src="https://placehold.co/200"), cls='myclass')))
print(to_xml(page))

<!doctype html></!doctype>

<html>
  <head>
    <title>Some page</title>
  </head>
  <body>
    <div class="myclass">
Some text, 
      <a href="https://example.com">A link</a>
      <img src="https://placehold.co/200">
    </div>
  </body>
</html>



In [ ]:
show(page)

If that `import *` worries you, you can always import only the tags you need. 

FastHTML is smart enough to know about fastcore.xml, and so you don't need to use the `to_xml` function to convert your XT objects to HTML. You can just return them as you would any other python object. For example, if we modify our previous example to use fastcore.xml, we can return an XT object directly:

In [ ]:
app = FastHTML()

@app.get("/")
def home():
    return Div(H1('Hello, World'), P('Some text'), P('Some more text'))

This will render the HTML in the browser.

For debugging, you can right-click on the rendered HTML in the browser and select "Inspect" to see the underlying HTML that was generated. There you'll also find the 'network' tab, which shows you the requests that were made to render the page. Refresh and look for the request to `127.0.0.1` - and you'll see it's just a `GET` request to `/`, and the response body is the HTML you just returned.

You can also use Starlette's `TestClient` to try it out in a notebook: 

In [ ]:
from starlette.testclient import TestClient
client = TestClient(app)
r = client.get("/")
r.text

'<!doctype html></!doctype>\n\n<html>\n  <head>\n    <title>FastHTML page</title>\n    <meta charset="utf-8"></meta>\n    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>\n    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>\n    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>\n    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>\n  </head>\n  <body>\n<div>\n  <h1>Hello, World</h1>\n  <p>Some text</p>\n  <p>Some more text</p>\n</div>\n  </body>\n</html>\n'

FastHTML wraps things in an Html tag if you don't do it yourself (unless the request comes from htmx, in which case you get the element directly). 
See the section 'XT objects and HTML' for more on creating custom components or adding HTML rendering to existing python objects.
To give the page a non-default title, return a Title before your main content:

In [ ]:
app = FastHTML()

@app.get("/")
def home():
    return Title("Page Demo"), Div(H1('Hello, World'), P('Some text'), P('Some more text'))

client = TestClient(app)
print(client.get("/").text)

<!doctype html></!doctype>

<html>
  <head>
    <title>Page Demo</title>
    <meta charset="utf-8"></meta>
    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>
    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>
  </head>
  <body>
<div>
  <h1>Hello, World</h1>
  <p>Some text</p>
  <p>Some more text</p>
</div>
  </body>
</html>



We'll use this pattern often in the examples to follow.

## Defining Routes



The HTTP protocol defines a number of methods ('verbs') to send requests to a server. The most common are GET, POST, PUT, DELETE, and HEAD. We saw 'GET' in action before - when you navigate to a URL, you're making a GET request to that URL. We can do different things on a route for different HTTP methods. For example:

```python
@app.route("/", methods='get')
def home():
    return H1('Hello, World')

@app.route("/", methods=['post', 'put'])
def post():
    return "got a post or put request"
```

This says that when someone navigates to the root URL "/" (i.e. sends a GET request), they will see the big "Hello, World" heading. When someone submits a POST or PUT request to the same URL, the server should return the string "got a post or put request".

Aside: You can test the POST request with `curl -X POST http://127.0.0.1:8000 -d "some data"`. This sends some data to the server, you should see the response "got a post or put request" printed in the terminal.

There are a few other ways you can specify the route+method - FastHTML has `.get`, `.post`, etc. as shorthand for `route(..., methods=['get'])`, etc.

In [ ]:
@app.get("/")
def my_function():
    return "Hello World from a get request"

 Or you can use the `@app.route` decorator without a method but specify the method with the name of the function. For example:

In [ ]:
@app.route("/")
def post():
    return "Hello World from a post request"

In [ ]:
client.post("/").text

'Hello World from a post request'

You're welcome to pick whichever style you prefer. Using routes let's you show different content on different pages - '/home', '/about' and so on. You can also respond differently to different kinds of requests to the same route, as we shown above. You can also pass data via the route:

In [ ]:
@app.get("/greet/{nm}")
def greet(nm:str):
    return f"Good day to you, {nm}!"

client.get("/greet/dave").text

'Good day to you, dave!'

More on this in the 'More on Routing and Requests' section, which goes deeper into the different ways to get information from a request.

## Styling Basics

Plain HTML probably isn't quite what you imagine when you visualize your beautiful web app. CSS is the go-to language for styling HTML. But again, we don't want to learn extra languages unless we absolutely have to! Fortunately, there are ways to get much more visually appealing sites by relying on the hard work of others, using existing CSS libraries. One of our favourites is [PicoCSS](https://picocss.com/). To add a CSS file to HTML, you can use the `<link>` tag. Since we typically want things like CSS styling on all pages of our app, FastHTML lets you add shared headers when you define your app. And it already has `picolink` defined for convenience. As per the [pico docs](https://picocss.com/docs), we put all of our content inside a `<main>` tag with a class of `container`: 


In [ ]:
from fasthtml import *
# App with custom styling to override the pico defaults
css = Style(':root { --pico-font-size: 100%; --pico-font-family: Pacifico, cursive;}')
app = FastHTML(hdrs=(picolink, css))

@app.route("/")
def get():
    return Title("Hello World"), Main(H1('Hello, World'), cls="container")

Aside: We're returning a tuple here (a title and the main page). This is needed to tell FastHTML to turn the main body into a full HTML page that includes the headers (including the pico link and our custom css) which we passed in.

You can check out the pico [examples](https://picocss.com/examples) page to see how different elements will look. If everything is working, the page should now render nice text with our custom font, and it should respect the user's light/dark mode preferences too.



If you want to [override the default styles](https://picocss.com/docs/css-variables) or add more custom CSS, you can do so by adding a `<style>` tag to the headers as shown above. So you are allowed to write CSS to your heart's content - we just want to make sure you don't necessarily have to! Later on we'll see examples using other component libraries and tailwind css to do more fancy styling things, along with tips to get an LLM to write all those fiddly bits so you don't have to.

## Web Page -> Web App

Showing content is all well and good, but we typically expect a bit more *interactivity* from something calling itself a web app! So, let's add a few different pages, and use a form to let users add messages to a list:

In [ ]:
app = FastHTML()
messages = ["This is a message, which will get rendered as a paragraph"]

@app.get("/")
def home():
    return Main(H1('Messages'), 
                *[P(msg) for msg in messages],
                A("Link to Page 2 (to add messages)", href="/page2"))

@app.get("/page2")
def page2():
    return Main(P("Add a message with the form below:"),
                Form(Input(type="text", name="data"),
                     Button("Submit"),
                     action="/", method="post"))

@app.post("/")
def add_message(data:str):
    messages.append(data)
    return home()

We re-render the entire homepage to show the newly added message. This is fine, but modern web apps often don't re-render the entire page, they just update a part of the page. In fact even very complicated applications are often implemented as 'Single Page Apps' (SPAs). This is where HTMX comes in.

## HTMX

[HTMX](https://htmx.org/) addresses some key limitations of HTML. In vanilla HTML, links can trigger a GET request to show a new page, and forms can send requests containing data to the server. A lot of 'Web 1.0' design revolved around ways to use these to do everything we wanted. But why should only *some* elements be allowed to trigger requests? And why should we refresh the *entire page* with the result each time one does? HTMX extends HTML to allow us to trigger requests from *any* element on all kinds of events, and to update a part of the page without refreshing the entire page. It's a powerful tool for building modern web apps.

It does this by adding attributes to HTML tags to make them do things. For example, here's a page with a counter and a button that increments it:

In [ ]:
app = FastHTML()

count = 0

@app.get("/")
def home():
    return Title("Count Demo"), Main(
        H1("Count Demo"),
        P(f"Count is set to {count}", id="count"),
        Button("Increment", hx_post="/increment", hx_target="#count", hx_swap="innerHTML")
    )

@app.post("/increment")
def increment():
    print("incrementing")
    global count
    count += 1
    return f"Count is set to {count}"

The button triggers a POST request to `/increment` (since we set `hx_post="increment"`), which increments the count and returns the new count. The `hx_target` attribute tells HTMX where to put the result. If no target is specified it replaces the element that triggered the request. The `hx_swap` attribute specifies how it adds the result to the page. Useful options are:

- *`innerHTML`*: Replace the target element's content with the result.
- *`outerHTML`*: Replace the target element with the result.
- *`beforebegin`*: Insert the result before the target element.
- *`beforeend`*: Insert the result inside the target element, after its last child.
- *`afterbegin`*: Insert the result inside the target element, before its first child.
- *`afterend`*: Insert the result after the target element.

You can also use an hx_swap of `delete` to delete the target element regardless of response, or of `none` to do nothing.

By default, requests are triggered by the “natural” event of an element - click in the case of a button (and most other elements). You can also specify different triggers, along with various modifiers - see the [HTMX docs](https://htmx.org/docs/#triggers) for more.

This pattern of having elements trigger requests that modify or replace other elements is a key part of the HTMX philosophy. It takes a little getting used to, but once mastered it is extremely powerful.

### Replacing Elements Besides the Target

Sometimes having a single target is not enough, and we'd like to specify some additional elements to update or remove. In these cases, returning elements with an id that matches the element to be replaces and `hx_swap_oob='true'` will replace those elements too. We'll use this in the next example to clear an input field when we submit a form.